<a href="https://colab.research.google.com/github/imammularif/Google-Colabs/blob/main/IF_BDDM_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A. Install pyspark

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 51.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=215c05d4c58149131fd7dc9731aa08619252401a6aacf80a2741283a7881a43b
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


# B. spark overview

3 Cara membuat tabel di SPARK:

- Spark dataframe
- Native Pandas
- Pandas API


Spark Dataframe

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
df = spark.createDataFrame([
    (1,"adi",90),
    (2,"budi",96),
    (3,"catur",76)
],schema=['no','nama','nilai'])

df.show()

+---+-----+-----+
| no| nama|nilai|
+---+-----+-----+
|  1|  adi|   90|
|  2| budi|   96|
|  3|catur|   76|
+---+-----+-----+



Pandas

In [ ]:
import pandas as pd

dfpd = pd.DataFrame({
    'no':[1,2,3],
    'nama':["adi","budi","catur"],
    'nilai':[90,96,76]
})
dfpdspark = spark.createDataFrame(dfpd)
dfpdspark.show()

+---+-----+-----+
| no| nama|nilai|
+---+-----+-----+
|  1|  adi|   90|
|  2| budi|   96|
|  3|catur|   76|
+---+-----+-----+



Pandas API

In [ ]:
import pyspark.pandas as ps

dfps = ps.DataFrame({
    'no':[1,2,3],
    'nama':["adi","budi","catur"],
    'nilai':[90,96,76]
})
dfps.to_spark().show()

+---+-----+-----+
| no| nama|nilai|
+---+-----+-----+
|  1|  adi|   90|
|  2| budi|   96|
|  3|catur|   76|
+---+-----+-----+



Export import

In [ ]:
df.write.csv('coba.csv',header=True)

In [ ]:
spark.read.csv('coba.csv',header=True).show()

+---+-----+-----+
| no| nama|nilai|
+---+-----+-----+
|  2| budi|   96|
|  3|catur|   76|
|  1|  adi|   90|
+---+-----+-----+



select and filter

In [ ]:
df.select("nama","nilai").show()

+-----+-----+
| nama|nilai|
+-----+-----+
|  adi|   90|
| budi|   96|
|catur|   76|
+-----+-----+



In [ ]:
df.filter(df.nilai > 90).show()

+---+----+-----+
| no|nama|nilai|
+---+----+-----+
|  2|budi|   96|
+---+----+-----+



SQL query

In [ ]:
df.createOrReplaceTempView("tabledf")
spark.sql("SELECT * from tabledf where nilai < 90").show()

+---+-----+-----+
| no| nama|nilai|
+---+-----+-----+
|  3|catur|   76|
+---+-----+-----+



# C. machine learning

klasifikasi

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd

data = load_iris()

df = pd.DataFrame(data.data,columns=data.feature_names)
df['target'] = data.target

dfml = spark.createDataFrame(df)
dfml.show()

+-----------------+----------------+-----------------+----------------+------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|
+-----------------+----------------+-----------------+----------------+------+
|              5.1|             3.5|              1.4|             0.2|     0|
|              4.9|             3.0|              1.4|             0.2|     0|
|              4.7|             3.2|              1.3|             0.2|     0|
|              4.6|             3.1|              1.5|             0.2|     0|
|              5.0|             3.6|              1.4|             0.2|     0|
|              5.4|             3.9|              1.7|             0.4|     0|
|              4.6|             3.4|              1.4|             0.3|     0|
|              5.0|             3.4|              1.5|             0.2|     0|
|              4.4|             2.9|              1.4|             0.2|     0|
|              4.9|             3.1|              1.

Vector Assembly

In [ ]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(inputCols=['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)'],
                outputCol='features')
dfnew = va.transform(dfml)
dfnew.show()

+-----------------+----------------+-----------------+----------------+------+-----------------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|         features|
+-----------------+----------------+-----------------+----------------+------+-----------------+
|              5.1|             3.5|              1.4|             0.2|     0|[5.1,3.5,1.4,0.2]|
|              4.9|             3.0|              1.4|             0.2|     0|[4.9,3.0,1.4,0.2]|
|              4.7|             3.2|              1.3|             0.2|     0|[4.7,3.2,1.3,0.2]|
|              4.6|             3.1|              1.5|             0.2|     0|[4.6,3.1,1.5,0.2]|
|              5.0|             3.6|              1.4|             0.2|     0|[5.0,3.6,1.4,0.2]|
|              5.4|             3.9|              1.7|             0.4|     0|[5.4,3.9,1.7,0.4]|
|              4.6|             3.4|              1.4|             0.3|     0|[4.6,3.4,1.4,0.3]|
|              5.0|           

split data

In [ ]:
train, test = dfnew.randomSplit([0.7,0.3],seed = 123)


training

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features',labelCol='target')

modellr = lr.fit(train)

In [ ]:
modellr.summary.accuracy

1.0

testing

In [ ]:
pred = modellr.transform(test)
pred.show()

+-----------------+----------------+-----------------+----------------+------+-----------------+--------------------+--------------------+----------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|         features|       rawPrediction|         probability|prediction|
+-----------------+----------------+-----------------+----------------+------+-----------------+--------------------+--------------------+----------+
|              4.4|             3.0|              1.3|             0.2|     0|[4.4,3.0,1.3,0.2]|[215.707349787378...|[1.0,3.7712967637...|       0.0|
|              4.6|             3.2|              1.4|             0.2|     0|[4.6,3.2,1.4,0.2]|[215.826867809311...|[1.0,2.1087204706...|       0.0|
|              4.7|             3.2|              1.3|             0.2|     0|[4.7,3.2,1.3,0.2]|[215.383815494839...|[1.0,5.9822545819...|       0.0|
|              4.8|             3.0|              1.4|             0.3|     0|[4.8,3.0,1.4,0.3]|[187

evaluasi

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
ev = MulticlassClassificationEvaluator()
ev.setPredictionCol('prediction')
ev.setLabelCol('target')
ev.evaluate(pred)

0.9349384447561165